In [1]:
import os

import pandas as pd

In [2]:
# general configs
pd.set_option("display.max_columns", 500)

In [62]:
# setting configs and constants
# path to raw dataset
file_cond = "../../data/raw/CONDICIONES.csv"
file_inv = "../../data/raw/INVESTIGACION.csv"
files = [file_cond, file_inv]
for file in files:
    if not(os.path.exists(file) and os.path.isfile(file)):
        raise Exception("File {} doesn't exists.".format(file))

In [63]:
df_inv = pd.read_csv(file_inv)
# df_inv.drop(columns="index", inplace=True)
df_inv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22468 entries, 0 to 22467
Data columns (total 43 columns):
COD_CIA                22468 non-null int64
COD_RAMO               22468 non-null int64
NUM_SINI               22468 non-null int64
NRO_EXPED              22468 non-null int64
NRO_ORDEN_INV          22468 non-null int64
NRO_INVEST             22468 non-null int64
COD_TALLER             0 non-null float64
IMPORTE                22467 non-null object
FECHA_INI_INVEST       22468 non-null object
FECHA_ENT_INVEST       22468 non-null object
FECHA_FIN_INVEST       13711 non-null object
FECHA_MOD_INVEST       14103 non-null object
EXISTE_INVEST          22468 non-null object
COD_INVEST_INT         129 non-null object
COD_DOCUM_INVEST1      17431 non-null object
COD_INVEST1            17431 non-null float64
NOM_INVEST1            17431 non-null object
COD_DOCUM_INVEST2      4 non-null object
COD_INVEST2            4 non-null float64
NOM_INVEST2            4 non-null object
COD_DOCUM_IN

/home/voolkia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [64]:
df_cond = pd.read_csv(file_cond)
# df_cond.drop(columns="index", inplace=True)
df_cond.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10230858 entries, 0 to 10230857
Data columns (total 6 columns):
NUM_SECU_EXPED     int64
NRO_ORDEN_EXP      int64
CONDICION          object
VALOR_CONDICION    float64
FEC_ACT            object
USR_ACT            object
dtypes: float64(1), int64(2), object(3)
memory usage: 468.3+ MB


In [65]:
col = "NUM_SECU_EXPED"
common = set(df_cond[col].unique()).intersection(set(df_inv[col].unique()))
len(common), len(common)/len(df_cond[col].unique()) * 100, len(common)/len(df_inv[col].unique()) * 100

(12742, 1.035080807301292, 97.32661167124962)

In [66]:
idx = set(df_inv["NUM_SECU_EXPED"].unique()) - set(df_cond["NUM_SECU_EXPED"].unique()) 
len(idx)

350

In [11]:
len(set(df_cond["NUM_SECU_EXPED"].unique()) - set(df_inv["NUM_SECU_EXPED"].unique()))

1218273

In [67]:
# getting NUM_SECU_EXPED in common
print(df_inv["NUM_SECU_EXPED"].nunique(), df_cond["NUM_SECU_EXPED"].nunique())

in_both = set(df_cond["NUM_SECU_EXPED"].unique()).intersection(set(df_inv["NUM_SECU_EXPED"].unique()))
len(in_both)

13092 1231015


12742

In [68]:
inv_not_in_cond = df_inv[~df_inv["NUM_SECU_EXPED"].isin(in_both)]
print("NUM_SECU_EXPED unique:{}".format(inv_not_in_cond["NUM_SECU_EXPED"].nunique()))
inv_not_in_cond["EXISTE_FRAUDE"].value_counts()

NUM_SECU_EXPED unique:350


N    552
S    136
Name: EXISTE_FRAUDE, dtype: int64

In [33]:
inv_not_in_cond.to_csv("investigaciones sin aparecer en condiciones.csv")

In [69]:
# doble checking
set(idx) - set(inv_not_in_cond["NUM_SECU_EXPED"].unique())

set()

# Checking date

In [53]:
# check in DSS_SINI_AUTOS_ID
# creating the path
path = "../../data/raw"
# file
file_sini = os.path.join(path, "DSS_SINI_AUTOS_ID.csv")
if not(os.path.exists(file_sini) and os.path.isfile(file_sini)):
    raise Exception("File {} doesn't exists.".format(file_sini))

In [70]:
usecols = ["FECHA_SINI", "NUM_SECU_EXPED"]
df_sini = pd.read_csv(file_sini, usecols=usecols)

/home/voolkia/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [71]:
# getting the dates of the records only in investigacion
only_in_inv = df_sini[df_sini["NUM_SECU_EXPED"].isin(idx)]
only_in_inv["FECHA_SINI"] = pd.to_datetime(only_in_inv["FECHA_SINI"], format="%d/%m/%Y")
print(len(not_in_inv))
only_in_inv["FECHA_SINI"].value_counts()

98


/home/voolkia/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


2017-12-12    17
2018-11-02    11
2018-11-07    11
2017-04-20     9
2017-03-10     7
2017-03-13     6
2018-10-21     6
2018-08-23     6
2018-06-11     5
2017-09-03     5
2018-10-14     5
2018-07-02     5
2018-08-10     3
2019-05-07     2
Name: FECHA_SINI, dtype: int64

In [72]:
only_in_inv["NUM_SECU_EXPED"].nunique()

14

In [74]:
only_in_inv_uniq = only_in_inv.drop_duplicates(subset=["NUM_SECU_EXPED"], keep="first")
len(only_in_inv_uniq)
only_in_inv_uniq["FECHA_SINI"].value_counts()

2018-06-11    1
2017-09-03    1
2018-11-07    1
2017-12-12    1
2017-03-10    1
2017-03-13    1
2018-07-02    1
2018-08-10    1
2018-08-23    1
2018-10-21    1
2018-11-02    1
2017-04-20    1
2018-10-14    1
2019-05-07    1
Name: FECHA_SINI, dtype: int64

In [61]:
only_in_inv_uniq.sort_values(by="FECHA_SINI").to_csv("investigaciones sin aparecer en condiciones en la tabla DSS_SINIESTROS.csv")

In [27]:
# checking distrib of EXISTE_FRAUDE
df_inv[df_inv["NUM_SECU_EXPED"].isin(not_in_inv[not_in_inv["FECHA_SINI"] >= "2018-01-01"]["NUM_SECU_EXPED"])]["EXISTE_FRAUDE"].value_counts(dropna=False)

False    9
Name: EXISTE_FRAUDE, dtype: int64

In [33]:
# setting configs and constants
# path to raw dataset
PATH = "../../data/interim/1. col_red"
FILENAME = "CONDICIONES-red_col.feather"
RAW_FILE = os.path.join(PATH, FILENAME)

if not(os.path.exists(RAW_FILE) and os.path.isfile(RAW_FILE)):
    raise Exception("File doesn't exists.")

In [34]:
df = pd.read_feather(RAW_FILE)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10230858 entries, 0 to 10230857
Data columns (total 5 columns):
CONDICION          object
FEC_ACT            datetime64[ns]
NUM_SECU_EXPED     int64
USR_ACT            object
VALOR_CONDICION    float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(2)
memory usage: 390.3+ MB


In [35]:
df[df["NUM_SECU_EXPED"].isin(idx)]

,CONDICION,FEC_ACT,NUM_SECU_EXPED,USR_ACT,VALOR_CONDICION
